In [1]:
# django shell 환경설정
import os
import django
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

django.setup()

# 페이징 처리
-  대량의 데이터를 여러 페이지로 나눠서 출력하는 것.
-  Django에서는 Paginator와 Page 클래스를 통해 처리한다.
  
## Paginator 클래스
- 전체 페이징 처리를 관리하는 클래스
- 전체 데이터관련 정보, 각 페이지당 보여줄 데이터의 정보 등을 제공

## Page 클래스
- 한페이지에대한 데이터를 관리
- Paginator를 통해서 생성.
    - `Pagenator객체.page(페이지 번호)`
- iterable 타입. 페이지에 속한 데이터들을 제공
- Page객체.object_list 속성: 페이지가 가진 데이터들을 List로 반환

In [6]:
txt = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
dataset = list(txt)
len(txt), len(dataset)
# dataset

(62, 62)

In [7]:
from django.core.paginator import Paginator

# Paging관리 -> Paginator 객체를 생성 -> 전체 데이터를 다 넣어서 생성.
pn = Paginator(dataset, 5) # (전체 데이터들, 페이지당 데이터개수)

In [10]:
print("전체 데이터 개수:", pn.count)
print("총 페이지수:", pn.num_pages)
print("시작페이지번호, 끝 페이지 번호:", pn.page_range)

전체 데이터 개수: 62
총 페이지수: 13
시작페이지번호, 끝 페이지 번호: range(1, 14)


In [12]:
for page in pn.page_range:
    print(page, end="\t")

1	2	3	4	5	6	7	8	9	10	11	12	13	

In [11]:
# 특정 Page값들을 조회 -> Page객체
page1 = pn.page(1) # 1 page 조회 -> 시작페이지
print(type(page1))
page1

<class 'django.core.paginator.Page'>


<Page 1 of 13>

In [12]:
page10 = pn.page(10) # 중간 페이지
page10

<Page 10 of 13>

In [13]:
page13 = pn.page(13) # 마지막 페이지
page13

<Page 13 of 13>

In [15]:
# pn.page(100) # 없는 페이지. EmptyPage Exception발생.

In [20]:
# Page객체가 가진 데이터들을 조회 -> Page: Iterable, Subscriptable
# page1[1]
for data in page1:
    print(data, end=", ")
print()
for data in page10:
    print(data, end=", ")
print()
for data in page13:
    print(data, end=", ")

0, 1, 2, 3, 4, 
J, K, L, M, N, 
Y, Z, 

In [21]:
# page의 데이터를 list로 변환
page1.object_list

['0', '1', '2', '3', '4']

## 이전/다음 페이지가 있는지 
- `Page객체.has_previous()` / `Page객체.has_next()`
- 1페이지: 이전페이지? X, 다음페이지? O
- 중간 페이지: 이전페이지? O, 다음페이지? O
- 마지막 페이지: 이전? O, 다음페이지? X

In [22]:
page1.has_previous(), page1.has_next()

(False, True)

In [23]:
page10.has_previous(), page10.has_next()

(True, True)

In [24]:
page13.has_previous(), page13.has_next()

(True, False)

## 이전/다음페이지 번호 조회
- **Page객체.number:** 현재 페이지 번호
- **Page객체.previous_page_number():** 이전페이지 번호 조회
- **page객체.next_page_number():** 다음페이지 번호 조회

In [27]:
page1.number, page10.number, page13.number

(1, 10, 13)

In [29]:
# 이전 페이지 번호
page10.previous_page_number()
page13.previous_page_number()

12

In [31]:
if page1.has_previous():
    print(page1.previous_page_number()) # 이전페이지가 없을 경우(1page에서 조회) EmptyPage 예외발생

In [34]:
# 다음 페이지 번호
page1.next_page_number()
page10.next_page_number()

11

In [36]:
if page13.has_next():
    print(page13.next_page_number())# 다음 페이지가 없을 경우(마지막 page에서 조회) EmptyPage 예외발생

## 각 페이지별 데이터를 출력(조회)

In [48]:
# 한페지당 K개의 데이터를 출력
# ------N 페이지-------
# a, b, c, d, e, ...
paginate_by = 10 # k개
# Paginator객체 생성
pn = Paginator(dataset, paginate_by)
# pn.page_range
# 반복문으로 각 페이지의 데이터를 출력
for page_num in pn.page_range:
    page = pn.page(page_num)
    print(f"--------------page:{page.number}------------------")
    for data in page:
        print(data, end=", ")
    print()

--------------page:1------------------
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
--------------page:2------------------
a, b, c, d, e, f, g, h, i, j, 
--------------page:3------------------
k, l, m, n, o, p, q, r, s, t, 
--------------page:4------------------
u, v, w, x, y, z, A, B, C, D, 
--------------page:5------------------
E, F, G, H, I, J, K, L, M, N, 
--------------page:6------------------
O, P, Q, R, S, T, U, V, W, X, 
--------------page:7------------------
Y, Z, 


# 현재 페이지(요청페이지)가 속한 page 그룹의 (page_range)에서의 시작 index와 끝 index를 조회

In [55]:
for p in pn.page_range[0:5]:
    print(p, end=', ')

1, 2, 3, 4, 5, 

In [56]:
pn = Paginator(dataset, 5)
r = pn.page_range
r

range(1, 14)

In [ ]:
# 하나의 page group에 3페이지씩 묶는 경우.
# index: 0 ~ 2, 3 ~ 5, 6 ~ 8, ...
print(r[0:3]) # 현재 page가 1, 2, 3 일 때, 그 페이지들이 속한 페이지의 시작/끝 index
print(r[3:6]) # 현재 page가 4, 5, 6 일 때, 그 페이지들이 속한 페이지의 시작/끝 index
print(r[6:9]) # 현재 page가 7, 8, 9 일 때, 그 페이지들이 속한 페이지의 시작/끝 index

range(1, 4)
range(4, 7)
range(7, 10)


In [ ]:
int(0.99999) # 내림

0

In [72]:
current_page = 20 # 현재 페이지 번호
page_group_count = 10 # 페이지 그룹당 묶을 페이지 개수 (1 page group: 1, 2, 3, 4, 5 ->의 index를 조회)

start_index = int((current_page - 1)/page_group_count) * page_group_count
end_index = start_index + page_group_count
print(start_index, end_index)

10 20


In [73]:
for p in pn.page_range[start_index:end_index]:
    print(p, end=',')

11,12,13,

In [74]:
pn.page_range

range(1, 14)

## Question, Choice Dummy 데이터 추가

In [75]:
from polls.models import Question, Choice

In [76]:
for i in range(1, 403): # 402개 추가
    q = Question(question_text=f"질문 - {i}")
    q.save()

In [77]:
# Question 개수 확인
cnt = Question.objects.all().count()
cnt

402

In [78]:
start_id = Question.objects.all()[0].pk
start_id

10

In [80]:
# 각 문제당 보기 4개씩 추가
import random
for q in Question.objects.all():
    for i in range(4):  # 4: 보기개수
        choice_text = f"{i}번째 보기입니다."
        votes = random.randint(0, 150)
        c = Choice(choice_text=choice_text, votes=votes, question=q)
        c.save()

In [81]:
Choice.objects.all().count()

1608